# Field Goals: An Observational Study

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

from matching.cem import CEM
from matching.imbalance import imbalance, get_imbalance_params
from util.evaluation import odds_ratios, binned_residuals, plot_binned_residuals, to_frame
from util.metrics import correlations
from util.fg_data import clean, get_data

from collections import OrderedDict
from tqdm import tqdm
import pandas as pd
import numpy as np
import mysql.connector
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import datetime as dt

plt.style.use('seaborn-darkgrid')
sns.set_palette('colorblind')

## Preprocessing

In [ ]:
# load and clean the data
cnx = mysql.connector.connect(user='root', password='mOntie20!mysql', host='127.0.0.1', database='nfl')
df = get_data(cnx, 'g.seas<=2019', xp=False, base='raw_6_cat')
df = clean(df, dropna=False)
df = df.drop(['fkicker', 'home_team', 'stadium', 'team', 'XP', 'humid', 'kicks', 'age', 'form'], axis=1)
df['year'] = df['year']-df['year'].min()
df.info()

In [ ]:
binary = ['altitude', 'iced', 'turf', 'postseason', 'away_game', 'precipitation']
continuous = ['distance', 'year', 'seasons', 'temperature', 'wind', 'pressure']

In [ ]:
df.head()

## Correlations

In [ ]:
df_corr = correlations(df, continuous, binary)
df_corr
# print(df_corr[abs(df_corr['corr'])>0.3].sort_values('corr').round(2).to_latex())
# print(df.loc[:,continuous].corr('spearman').round(2).to_latex())

In [ ]:
# sns.set(font_scale=1)
# ax = sns.pairplot(df.loc[:,continuous].sample(frac=0.5), diag_kind='hist', plot_kws=dict(s=10, edgecolor="b", linewidth=1))

## Finding H

In [ ]:
rows = []
cont_bins = range(1, 10)

for h, treatment in tqdm(itertools.product(cont_bins,binary), total=len(binary)*len(cont_bins)):
    bins = get_imbalance_params(df.drop([treatment, 'good'], axis=1),
                                'l1', continuous, h)
    l1 = imbalance(df.drop('good', axis=1).astype(int), treatment, 'l1', bins)
    rows.append({'L1':l1, 'treatment':treatment, 'H':h})

In [ ]:
imb = pd.DataFrame.from_records(rows)
imb = imb.loc[imb['L1']<1, :]

ax = sns.lineplot(x='H', y='L1', data=imb, hue='treatment', style='treatment', markers=True)
ax.set_title('L1 profile of raw data', size=16)
ax.set_xlabel('H', size=16)
ax.set_ylabel('L1    ', size=16, rotation='horizontal')
ax.tick_params(labelsize=12, size=2)
plt.savefig('../images/L1.png')

## Without Matching

#### Imbalance

In [ ]:
def pp(val):
    # for annotating regression coefficients by significance
    if val<0.01:
        return '***'
    elif val<0.05:
        return '**'
    elif val<0.1:
        return '*'
    return ''

In [ ]:
multi = {}
imb = {}
for t in binary:
    cem = CEM(df, t, 'good', continuous, H=4)
    multi[t] = cem.preimbalance

    ub = cem.univariate_imbalance()
    imb[t] = ub['imbalance'].round(3).astype(str)+ub['P>|z|'].apply(pp) 

df_imb = pd.DataFrame.from_dict(imb, orient='columns')
df_imb = df_imb.append(pd.DataFrame(multi, index=['multivariate']))

In [ ]:
print(df_imb.to_latex())

#### Regression

* df_unmatched is all covariate estimates for all models.
* results_unmatched is a dict of all statsmodels.Results.

In [ ]:
formulas_simple = {t:f'good ~ {t}' for t in df.drop('good', axis=1).columns}
formula_main = {'main effects':'good ~ '+ ' + '.join(df.drop('good', axis=1).columns)}
formula_interaction = {'interactions':'good ~ '+ ' + '.join(df.drop('good', axis=1).columns) + f'+ iced*pressure + altitude*distance'}
formula_full = {'full':'good ~ ' + ' + '.join(df.drop(['good'], axis=1).columns) + f'+ np.log(wind+1) + np.log(seasons) + np.log(distance) + iced*pressure + altitude*distance'}
formulas = {**formulas_simple, **formula_main, **formula_interaction, **formula_full}

In [ ]:
df_unmatched = pd.DataFrame(columns=['coef', 'P>|z|', 'bse', 'model'])
results_unmatched = {}
for name, formula in formulas.items():
    glm = smf.glm(formula, data=df, family=sm.families.Binomial())
    result = glm.fit(method='bfgs', maxiter=10000)
    results_unmatched[name] = result
    model = name if name not in df.columns else 'simple'
    df_unmatched = pd.concat((df_unmatched, pd.DataFrame({'coef':result.params, 'P>|z|':result.pvalues, 'bse': result.bse, 'model':model})))

df_unmatched.index.rename('covariate', inplace=True)
df_unmatched = df_unmatched.reset_index('covariate')
results_unmatched = pd.Series(results_unmatched, name='model')

In [ ]:
df_unmatched.head()

In [ ]:
# get the order of covariates so the table looks pretty
order_ = OrderedDict()
for cov in df_unmatched['covariate']:
    if cov != 'Intercept':
        order_[cov] = 1

order_ = list(order_.keys())
order_

In [ ]:
df_um = df_unmatched.loc[df_unmatched['covariate']!='Intercept', :]
df_um['coef'] = df_um['coef'].round(3).astype(str) + df_um['P>|z|'].apply(pp)
df_um = df_um.loc[:,['covariate', 'coef', 'model']]
print(df_um.pivot('covariate', columns='model', values='coef').loc[order_, ['simple', 'main effects', 'interactions', 'full']].to_latex())

In [ ]:
# SATE estimates for each model for each possible treatment
from scipy.stats import norm
sates = []
for t in binary:
    df_treated = df.copy()
    df_control = df.copy()
    df_treated[t] = 1
    df_control[t] = 0

    for name, result in results_unmatched.items():
        if (name in continuous) or (name in binary and name!=t):
            continue
        # calculate outcomes under treatment or not
        treated_outcome = result.predict(df_treated)
        control_outcome = result.predict(df_control)
        t_mean = treated_outcome.mean()
        c_mean = control_outcome.mean()
        sate = t_mean - c_mean

        # significance test
        pooled = (t_mean+c_mean)/2
        pooled_se = np.sqrt(2*pooled*(1-pooled)/len(df))
        z = sate/pooled_se
        p = norm.sf(abs(z))

        sates.append({'treatment':t, 'model':name if name not in binary else 'simple', 'SATE':str(round(sate,3))+pp(p)})
df_sate = pd.DataFrame.from_records(sates).set_index(['treatment', 'model'])
print(df_sate.round(3).to_latex())
# df_sate.round(3)

Conditional treatment effects (heterogeneous)

In [ ]:
def CATE(result, T, X, X_val):
    # we asssume the rows of result contain the estimates for covariates that contain the binary treatment variable
    result = result.copy()
    result['val'] = result['covariate'].str.replace(':', '*').apply(lambda x: eval(x, {T:1, X:X_val, 'np':np}))
    result['val*beta'] = result['val']*result['coef']
    result.set_index('covariate', inplace=True)
    cate = result['val*beta'].sum()
    variance = {x: vc.loc[x,x] for x in result.index}
    covariance = {(x,y): vc.loc[x, y] for x, y in itertools.combinations(result.index, 2)}
    variance_terms = sum([v*result.loc[k,'val']**2 for k, v in variance.items()])
    covariance_terms = sum([2*result.loc[x,'val']*result.loc[y, 'val']*cv for (x,y), cv in covariance.items()])
    se = np.sqrt(variance_terms + covariance_terms)
    return cate, se

In [ ]:
pairs = (('iced', 'pressure'), ('altitude', 'distance'))
models = ('interactions', 'full')
for (T, M), model in itertools.product(pairs, models):
    vals = range(df[M].min(), df[M].max()+1)

    result = df_unmatched.loc[(df_unmatched['model']==model) & ([T in r for r in df_unmatched['covariate']]), :]

    ATE, SE = zip(*[CATE(result, T, M, d) for d in vals])

    fig, axes = plt.subplots(2,1)
    sns.distplot(df[M], kde=False, ax=axes[1])
    ax = sns.lineplot(x=vals, y=ATE, ax=axes[0])
    lb = np.array(ATE) - 1.96*np.array(SE)
    ub = np.array(ATE) + 1.96*np.array(SE)
    axes[0].fill_between(vals,lb,ub, alpha=0.3)
    ax.set_title(f'Marginal effect of {T} with {M}')
    axes[1].set_xlabel(M)
    ax.set_ylabel(f'Marginal effect of {T}')
    axes[1].set_ylabel('Count')
    ax.set_xticklabels([])
    ax.hlines(0,df[M].min(), df[M].max(), colors='r', alpha=0.5, linestyles='dashed')
    plt.savefig(f'../images/unmatched_{T}/{M}_{model}.png')
    plt.cla()
    plt.clf()

In [ ]:
pres = df['pressure'].value_counts().sort_index(ascending=True).round(3).to_frame()
pres.index.rename('Level', inplace=True)
pres = pres.rename(columns={'pressure':'Count'})
print(pres.to_latex())

### Analysis of Residuals

In [ ]:
# models = ('main effects', 'interactions', 'full')
models = formulas.keys()
ons = ['pred']+continuous
for model, on in itertools.product(models, ons):
    resid = binned_residuals(results_unmatched[model], df.drop('good', axis=1), df['good'], bins=None, on=on)
    pct = round(resid['inside'].mean()*100,1)
    ax = plot_binned_residuals(resid, on=on)
    ax.set_title(f'Binned residuals on {on} for {model} model ({pct}% inside)')
    model = '_'.join(model.split())
    plt.savefig(f'../images/unmatched_residuals/{model}_{on}.png')
    plt.cla()

## With Matching

### Coarsening

In [ ]:
def var_plot(data, var, cuts=3, func=pd.cut, bins=None, **kwargs):
    ax = sns.distplot(data[var], kde=False, bins=bins)    
    out, bins = func(data[var], cuts, retbins=True, **kwargs)
    for bin_ in bins:
        plt.axvline(bin_, color='r', linestyle='dashed', linewidth=2)
    return ax, out

In [ ]:
coarsening = {
    'distance': {'bins':range(15,86,10), 'cut': 'cut'}, # relax over
    'year': {'bins': 5, 'cut': 'cut'}, # uniform so qcut=cut
    'seasons': {'bins': [0,4,10,18,30], 'cut': 'cut'},
    'temperature': {'bins': [-25,-5,5,15,25,35,45], 'cut': 'cut'},
    'wind': {'bins': [-0.1,12,25,41], 'cut': 'cut'},
    'pressure': {'bins': 3, 'cut': 'cut'},
    'postseason': {'bins': 2, 'cut': 'cut'},
    'iced': {'bins': 2, 'cut': 'cut'},
    'precipitation': {'bins': 2, 'cut': 'cut'},
    'altitude': {'bins': 2, 'cut': 'cut'},
    'turf': {'bins': 2, 'cut': 'cut'},
    'away_game': {'bins': 2, 'cut': 'cut'},
    'altitude': {'bins': 2, 'cut': 'cut'},
}

In [ ]:
for var, coarsen in coarsening.items():
    cut = coarsen['cut']
    div = 2 if var in ('wind', 'temperature') else 1
    ax, bins = var_plot(df, var, cuts=coarsen['bins'], bins=int(df[var].nunique()/div), func=eval(f'pd.{cut}'))
    ax.tick_params(size=4)
    ax.set_title(f'Coarsening of {var}')
    # bins.cat.categories
    plt.savefig(f'../images/coarsening/{var}.png')
    plt.cla()

#### Imbalance

In [ ]:
df_imb = None
multi = {}
n_matched = {}
t_matched = {}
uni = {}
for t in binary:
    cem = CEM(df, t, 'good', continuous, H=4)
    ub = cem.univariate_imbalance(coarsening)
    uni[t] = ub['imbalance'].round(3).astype(str)+ub['P>|z|'].apply(pp)
    is_matched_ = cem.match(coarsening)>0
    n_matched[t] = is_matched_.sum()
    t_matched[t] = (df.loc[is_matched_, t] == 1).sum()/(df[t]==1).sum()*100
    multi[t] = cem.imbalance(coarsening)

df_imb = pd.DataFrame.from_dict(uni, orient='columns')
df_imb = df_imb.append(pd.DataFrame(multi, index=['multivariate']))
df_imb = df_imb.append(pd.DataFrame(n_matched, index=['% matched'])/len(df)*100)
df_imb = df_imb.append(pd.DataFrame(t_matched, index=['% treatment matched']))

In [ ]:
print(df_imb.to_latex())

### Regressions

In [ ]:
weights = {t:CEM(df, t, 'good', continuous, H=4).match(coarsening) for t in binary}
drop = {'altitude': ['turf'], 'turf': ['altitude']}

In [ ]:
def get_formula(model, t):
    if model == 'simple':
        formula = f'good ~ {t}'
    elif model == 'main effects':
        formula = 'good ~ '+ ' + '.join(df.drop(['good']+drop.get(t, []), axis=1).columns)
    elif model == 'interactions' and t == 'turf':
        formula = 'good ~'+ ' + '.join(df.drop(['good']+drop.get(t, []), axis=1).columns) + ' + iced*pressure'
    elif model == 'interactions':
        formula = 'good ~'+ ' + '.join(df.drop(['good']+drop.get(t, []), axis=1).columns) + ' + iced*pressure + altitude*distance'
    elif model == 'full' and t == 'turf':
        formula = 'good ~'+ ' + '.join(df.drop(['good']+drop.get(t, []), axis=1).columns) +\
        '+ np.log(wind+1) + np.log(seasons) + iced*pressure'
    elif model == 'full':
        formula = 'good ~'+ ' + '.join(df.drop(['good']+drop.get(t, []), axis=1).columns) +\
        ' + np.log(wind+1) + np.log(seasons) + np.log(distance) + iced*pressure + altitude*distance'

    return formula

In [ ]:
df_matched = pd.DataFrame(columns=['coef', 'P>|z|', 'bse', 'model', 'treatment'])
results_matched = []
models = ['simple', 'main effects', 'interactions', 'full']

for t, model in itertools.product(binary, models):
    forumla = get_formula(model, t)
    w = weights[t]
    glm = smf.glm(formula,
                data=df[w > 0],
                family=sm.families.Binomial(),
                var_weights=w[w > 0])
    result = glm.fit(method='bfgs', maxiter=1000)
    results_matched.append({'model': model, 'result':result, 'treatment': t})
    df_matched = pd.concat((df_matched, pd.DataFrame({'coef':result.params, 
                                                      'P>|z|':result.pvalues, 
                                                      'bse': result.bse,
                                                      'model':model, 
                                                      'treatment':t})))

df_matched.index.rename('covariate', inplace=True)
df_matched.reset_index('covariate', inplace=True)
results_matched = pd.DataFrame.from_records(results_matched)

In [ ]:
df_matched.head(5)

In [ ]:
results_matched.head(5)

In [ ]:
def drop_unrelated(group):
    t = group['treatment'].iloc[0]
    group = group.loc[group['covariate'].str.contains(t, regex=False)]
    return group

In [ ]:
df_m = df_matched.loc[df_matched['covariate']!='Intercept', :]
df_m = df_m.groupby('treatment').apply(drop_unrelated).reset_index(drop=True)
df_m['coef'] = df_m['coef'].round(3).astype(str) + df_m['P>|z|'].apply(pp)
df_m = df_m.loc[:,['covariate', 'coef', 'model', 'treatment']]
df_piv = df_m.pivot_table(index=['treatment','covariate'], columns='model', values='coef', aggfunc=lambda x: x).loc[order_, ['simple','main effects', 'interactions', 'full']]
print(df_piv.to_latex())

SATE

In [ ]:
from scipy.stats import ttest_rel
sates = []

for t in binary:

    # num all units
    n = len(df)

    # num all treated and controls
    n_t = (df[t]==1).sum()
    n_c = (df[t]==0).sum()

    # num matched units
    m = (weights[t]>0).sum()

    # num matched treated and controls
    m_t = len(df.loc[(weights[t]>0) & (df[t]==1),:])
    m_c = len(df.loc[(weights[t]>0) & (df[t]==0),:])

    # matched and unmatched units
    df_m = df.loc[(weights[t]>0), :] # matched
    df_um = df.loc[(weights[t]==0), :] # unmatched

    # matched treated units (observed or counterfactual)
    df_m_treated = df_m.copy()
    df_m_treated[t] = 1

    # matched control units (observed or counterfactual)
    df_m_control = df_m.copy()
    df_m_control[t] = 0

    # unmatched treated units (observed or counterfactual)
    df_um_treated = df_um.copy()
    df_um_treated[t] = 1

    # unmatched control units (observed or counterfactual)
    df_um_control = df_um.copy()
    df_um_control[t] = 0

    for _, row in results_matched.iterrows():
        result = row['result']

        if row['treatment'] != t:
            continue

        # matched predictions
        treated_m = result.predict(df_m_treated)
        control_m = result.predict(df_m_control)

        # unmatched predictions
        treated_um = result.predict(df_um_treated)
        control_um = result.predict(df_um_control)

        # sate for matched and unmatched units and their counterfactuals
        sate_m = (treated_m-control_m).mean().round(3)
        sate_um = (treated_um-control_um).mean().round(3)

        # weighted sate combining matched and unmatched sates
        sate_w = round((sate_m*m + sate_um*(n-m))/n,3)

        # significance test for matched
        pooled_m = (treated_m.mean() + control_m.mean())/2
        pooled_var_m = 2*pooled_m*(1-pooled_m)/m
        pooled_se_m = np.sqrt(pooled_var_m)
        z_m = sate_m/pooled_se_m
        p_m = norm.sf(abs(z_m))

        # significance test for unmatched
        pooled_um = (treated_um.mean() + control_um.mean())/2
        pooled_var_um = 2*pooled_um*(1-pooled_um)/(n-m)
        pooled_se_um = np.sqrt(pooled_var_um)
        z_um = sate_um/pooled_se_um
        p_um = norm.sf(abs(z_um))

        # weighted significance test
        pooled_var_w = (pooled_var_m*m + pooled_var_um*(n-m))/n
        pooled_se_w = np.sqrt(pooled_var_w)
        z_w = sate_w/pooled_se_w
        p_w = norm.sf(abs(z_w))

        sates.append({'treatment':t, 'model':row['model'], 'local SATE':str(sate_m)+pp(p_m), 'unmatched SATE':str(sate_um)+pp(p_um), 'weighted SATE':str(sate_w)+pp(p_w)})
df_sate = pd.DataFrame.from_records(sates).set_index(['treatment', 'model'])

In [ ]:
print(df_sate.to_latex())

#### SATT

In [ ]:
from scipy.stats import ttest_rel
satts = []

for t in binary:

    # all units
    nT = (df[t]==1).sum()

    # matched and unmatched treated units
    df_t_m = df.loc[(weights[t]>0) & (df[t]==1), :] # matched
    df_t_um = df.loc[(weights[t]==0) & (df[t]==1), :] # unmatched

    # observed outcomes for matched and unmatched treated units
    observed_m = df_t_m['good']
    observed_um = df_t_um['good']
    mT = len(observed_m) # matched treated units

    # counterfactual data points for matched treated units
    df_cf_m = df_t_m.copy()
    df_cf_m[t] = 0

    # counterfactuals for unmatched treated units
    df_cf_um = df_t_um.copy()
    df_cf_um[t] = 0

    for _, row in results_matched.iterrows():
        result = row['result']

        if row['treatment'] != t:
            continue

        # true predictions
        true_m = result.predict(df_t_m)
        true_um = result.predict(df_t_um)

        # potential outcome for matched and unmatched treated unit counterfactuals
        potential_m = result.predict(df_cf_m)
        potential_um = result.predict(df_cf_um)

        # satt for matched and unmatched treated units and their counterfactuals
        satt_m = (true_m-potential_m).mean().round(3)
        satt_um = (true_um-potential_um).mean().round(3)

        # weighted satt combining matched and unmatched satts
        satt_w = round((satt_m*mT + satt_um*(nT-mT))/nT,3)

        # t-test for matched and unmatched treated units and their counterfactuals. THIS IS POTENTIALLY VERY WRONG!
        stat_m, p_m = ttest_rel(observed_m, potential_m)
        stat_um, p_um = ttest_rel(observed_um, potential_um)
        p_w = (p_m*mT + p_um*(nT-mT))/nT

        satts.append({'treatment':t, 'model':row['model'], 'local satt':str(satt_m)+pp(p_m), 'unmatched satt':str(satt_um)+pp(p_um), 'weighted satt':str(satt_w)+pp(p_w)})
df_satt = pd.DataFrame.from_records(satts).set_index(['treatment', 'model'])

In [ ]:
print(df_satt.to_latex())

Conditional Treatment Effect

In [ ]:
pairs = (('iced', 'pressure'), ('altitude', 'distance'))
models = ('interactions', 'full')
for (T, M), model in itertools.product(pairs, models):
    vals = range(df[M].min(), df[M].max()+1)

    result = df_matched.loc[(df_matched['model']==model) & (df_matched['treatment']==T) & ([T in r for r in df_matched['covariate']]), :]

    ATE, SE = zip(*[CATE(result, T, M, d) for d in vals])

    fig, axes = plt.subplots(2,1)
    sns.distplot(df.loc[weights[T]>0, M], kde=False, ax=axes[1])
    ax = sns.lineplot(x=vals, y=ATE, ax=axes[0])
    lb = np.array(ATE) - 1.96*np.array(SE)
    ub = np.array(ATE) + 1.96*np.array(SE)
    axes[0].fill_between(vals,lb,ub, alpha=0.3)
    ax.set_title(f'Marginal effect of {T} with {M}')
    axes[1].set_xlabel(M)
    ax.set_ylabel(f'Marginal effect of {T}')
    axes[1].set_ylabel('Count')
    ax.set_xticklabels([])
    ax.hlines(0,df[M].min(), df[M].max(), colors='r', alpha=0.5, linestyles='dashed')
    plt.savefig(f'../images/matched_{T}/{M}_{model}.png')
    plt.cla()
    plt.clf()

In [ ]:
pres = df.loc[weights['iced']>0,'pressure'].value_counts().sort_index(ascending=True).round(3).to_frame()
pres.index.rename('Level', inplace=True)
pres = pres.rename(columns={'pressure':'Count'})
print(pres.to_latex())

Residuals

In [ ]:
models = ('main effects', 'interactions', 'full')
for model, t in itertools.product(models, binary):
    on = 'pred'
    df_ = df.loc[weights[t]>0, :] # matched
    resid = binned_residuals(results_matched.loc[(results_matched['model']==model) & (results_matched['treatment']==t), 'result'].iloc[0], df_.drop('good', axis=1), df_['good'], bins=None, on=on)
    pct = round(resid['inside'].mean()*100,1)
    ax = plot_binned_residuals(resid, on=on)
    ax.set_title(f'Residuals for {model} model after matching on {t} ({pct}% inside)')
    model = '_'.join(model.split())
    plt.savefig(f'../images/matched_residuals/{model}_{t}_{on}.png')
    plt.show()